# 01 - Data Cleaning & Preprocessing

**Credit Risk Assessment Model**

This notebook handles:
- Loading raw Lending Club data
- Column selection (removing irrelevant/leakage columns)
- Creating the target variable
- Stratified sampling
- Data type conversions
- Missing value handling
- Initial feature engineering

---

## 1. Import Libraries

In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Load Raw Data

In [27]:
# Load data - adjust filename as needed
df = pd.read_csv('../data/raw/loan.csv', low_memory=False)

print(f"Original dataset shape: {df.shape}")
print(f"Total rows: {df.shape[0]:,}")
print(f"Total columns: {df.shape[1]}")
print(f"\nMemory usage: {df.memory_usage(deep=True).sum() / 1e9:.2f} GB")

Original dataset shape: (2260701, 151)
Total rows: 2,260,701
Total columns: 151

Memory usage: 6.28 GB


In [28]:
# Quick look at the data
df.head(3)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,...,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,...,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,...,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,truck driver,10+ years,MORTGAGE,63000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,home_improvement,NaN,605xx,IL,10.78,...,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Check loan status distribution
print("Loan Status Distribution:")
print(df['loan_status'].value_counts())

Loan Status Distribution:
loan_status
Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: count, dtype: int64


## 3. Column Selection

We keep only relevant columns and remove:
- **Identifiers**: id, member_id, url (no predictive value)
- **Leakage columns**: Post-loan information like total_pymnt, recoveries
- **High missing**: Columns with >40% missing values

In [30]:
# Columns to keep for modeling
keep_cols = [
    # Loan information
    'loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'sub_grade',
    
    # Borrower information
    'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
    'purpose', 'addr_state', 'dti',
    
    # Credit history
    'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high',
    'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
    'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
    
    # Additional useful features
    'collections_12_mths_ex_med', 'acc_now_delinq', 'tot_coll_amt',
    'tot_cur_bal', 'total_rev_hi_lim',
    
    # Target and date
    'loan_status', 'issue_d'
]

# Keep only columns that exist in the dataset
keep_cols = [col for col in keep_cols if col in df.columns]
df_clean = df[keep_cols].copy()

print(f"Columns retained: {len(keep_cols)}")
print(f"Dataset shape after column selection: {df_clean.shape}")

Columns retained: 32
Dataset shape after column selection: (2260701, 32)


## 4. Create Target Variable

**Target**: Loan Default (1 = Default, 0 = Fully Paid)

We exclude "Current" loans since we don't know their final outcome.

In [31]:
# Define default statuses
default_statuses = [
    'Charged Off', 
    'Default', 
    'Late (31-120 days)',
    'Late (16-30 days)',
    'Does not meet the credit policy. Status:Charged Off'
]

# Define completed loan statuses (exclude "Current")
completed_statuses = [
    'Fully Paid', 
    'Charged Off', 
    'Default', 
    'Late (31-120 days)', 
    'Late (16-30 days)',
    'Does not meet the credit policy. Status:Fully Paid',
    'Does not meet the credit policy. Status:Charged Off'
]

# Filter to completed loans only
df_clean = df_clean[df_clean['loan_status'].isin(completed_statuses)]

# Create binary target
df_clean['target'] = df_clean['loan_status'].isin(default_statuses).astype(int)

print(f"Dataset shape after filtering: {df_clean.shape}")
print(f"\nTarget Distribution:")
print(df_clean['target'].value_counts())
print(f"\nDefault Rate: {df_clean['target'].mean()*100:.2f}%")

Dataset shape after filtering: (1373915, 33)

Target Distribution:
target
0    1078739
1     295176
Name: count, dtype: int64

Default Rate: 21.48%


## 5. Stratified Sampling

Sample 250,000 rows while preserving the default rate ratio.

In [32]:
SAMPLE_SIZE = 250000

if len(df_clean) > SAMPLE_SIZE:
    print(f"Sampling {SAMPLE_SIZE:,} rows from {len(df_clean):,} total rows...")
    
    df_clean, _ = train_test_split(
        df_clean,
        train_size=SAMPLE_SIZE,
        stratify=df_clean['target'],
        random_state=42
    )
    
    print(f"\nAfter sampling: {df_clean.shape}")
    print(f"Default rate preserved: {df_clean['target'].mean()*100:.2f}%")
else:
    print(f"Dataset size ({len(df_clean):,}) is manageable, no sampling needed.")

Sampling 250,000 rows from 1,373,915 total rows...

After sampling: (250000, 33)
Default rate preserved: 21.48%


## 6. Clean Specific Columns

In [33]:
# Term: Extract numeric value (36 or 60 months)
df_clean['term'] = df_clean['term'].str.extract(r'(\d+)').astype(float)
print(f"Term values: {df_clean['term'].unique()}")

Term values: [36. 60.]


In [34]:
# Interest rate: Remove % symbol if present
if df_clean['int_rate'].dtype == 'object':
    df_clean['int_rate'] = df_clean['int_rate'].str.replace('%', '', regex=False).astype(float)
print(f"Interest rate range: {df_clean['int_rate'].min():.2f}% - {df_clean['int_rate'].max():.2f}%")

Interest rate range: 5.31% - 30.99%


In [35]:
# Employment length: Convert to numeric
emp_map = {
    '< 1 year': 0.5, '1 year': 1, '2 years': 2, '3 years': 3,
    '4 years': 4, '5 years': 5, '6 years': 6, '7 years': 7,
    '8 years': 8, '9 years': 9, '10+ years': 10
}
df_clean['emp_length_num'] = df_clean['emp_length'].map(emp_map)
print(f"Employment length - Missing: {df_clean['emp_length_num'].isnull().sum()}")

Employment length - Missing: 14737


In [36]:
# Earliest credit line: Convert to credit history in years
# Get original string values from df (before any conversion)
original_dates = df.loc[df_clean.index, 'earliest_cr_line'].copy()

# Parse with correct format: %b-%Y (4-digit year like 'Aug-2003')
df_clean['earliest_cr_line_dt'] = pd.to_datetime(original_dates, format='%b-%Y', errors='coerce')

print(f"Successfully parsed: {df_clean['earliest_cr_line_dt'].notna().sum()}")
print(f"Failed to parse: {df_clean['earliest_cr_line_dt'].isna().sum()}")

# Calculate credit history years (using 2018-12-31 as reference)
reference_date = pd.Timestamp('2018-12-31')
df_clean['credit_history_years'] = (reference_date - df_clean['earliest_cr_line_dt']).dt.days / 365

# Fill missing with median
median_history = df_clean['credit_history_years'].median()
df_clean['credit_history_years'] = df_clean['credit_history_years'].fillna(median_history)

print(f"Credit history range: {df_clean['credit_history_years'].min():.1f} - {df_clean['credit_history_years'].max():.1f} years")
print(f"Median credit history: {median_history:.1f} years")

Successfully parsed: 249995
Failed to parse: 5
Credit history range: 3.3 - 75.0 years
Median credit history: 18.4 years


In [37]:
# Revolving utilization: Clean if needed
if df_clean['revol_util'].dtype == 'object':
    df_clean['revol_util'] = df_clean['revol_util'].str.replace('%', '', regex=False).astype(float)
print(f"Revolving utilization range: {df_clean['revol_util'].min():.1f}% - {df_clean['revol_util'].max():.1f}%")

Revolving utilization range: 0.0% - 193.0%


## 7. Handle Missing Values

In [38]:
# Check missing values
missing = df_clean.isnull().sum()
missing_pct = (missing / len(df_clean) * 100).round(2)

missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct
}).sort_values('Missing %', ascending=False)

print("Columns with missing values:")
print(missing_df[missing_df['Missing Count'] > 0])

Columns with missing values:
                            Missing Count  Missing %
mths_since_last_record             207463      82.99
mths_since_last_delinq             125618      50.25
emp_length_num                      14737       5.89
emp_length                          14737       5.89
total_rev_hi_lim                    12883       5.15
tot_cur_bal                         12883       5.15
tot_coll_amt                        12883       5.15
revol_util                            164       0.07
dti                                    85       0.03
collections_12_mths_ex_med             30       0.01
inq_last_6mths                          5       0.00
earliest_cr_line                        5       0.00
delinq_2yrs                             5       0.00
open_acc                                5       0.00
acc_now_delinq                          5       0.00
total_acc                               5       0.00
pub_rec                                 5       0.00
earliest_cr_line_

In [39]:
# Fill numeric columns with median
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
df_clean[numeric_cols] = df_clean[numeric_cols].fillna(df_clean[numeric_cols].median())

print(f"Missing values after imputation: {df_clean[numeric_cols].isnull().sum().sum()}")

Missing values after imputation: 0


## 8. Feature Engineering

In [40]:
# Loan to income ratio
df_clean['loan_to_income'] = df_clean['loan_amnt'] / (df_clean['annual_inc'] + 1)

# Monthly debt burden (installment as % of monthly income)
df_clean['monthly_debt_burden'] = df_clean['installment'] / (df_clean['annual_inc']/12 + 1)

# FICO average
df_clean['fico_avg'] = (df_clean['fico_range_low'] + df_clean['fico_range_high']) / 2

# Grade numeric (A=7, B=6, ..., G=1)
grade_map = {'A': 7, 'B': 6, 'C': 5, 'D': 4, 'E': 3, 'F': 2, 'G': 1}
df_clean['grade_num'] = df_clean['grade'].map(grade_map)

print("New features created:")
print("- loan_to_income")
print("- monthly_debt_burden")
print("- fico_avg")
print("- grade_num")
print("- emp_length_num")
print("- credit_history_years")

New features created:
- loan_to_income
- monthly_debt_burden
- fico_avg
- grade_num
- emp_length_num
- credit_history_years


## 9. Final Dataset Summary

In [41]:
print("="*60)
print("FINAL DATASET SUMMARY")
print("="*60)
print(f"\nShape: {df_clean.shape}")
print(f"Rows: {df_clean.shape[0]:,}")
print(f"Columns: {df_clean.shape[1]}")
print(f"\nDefault Rate: {df_clean['target'].mean()*100:.2f}%")
print(f"\nTarget Distribution:")
print(df_clean['target'].value_counts())

FINAL DATASET SUMMARY

Shape: (250000, 40)
Rows: 250,000
Columns: 40

Default Rate: 21.48%

Target Distribution:
target
0    196289
1     53711
Name: count, dtype: int64


In [42]:
# List all columns
print(f"\nAll Columns ({len(df_clean.columns)}):")
for i, col in enumerate(df_clean.columns, 1):
    print(f"  {i}. {col}")


All Columns (40):
  1. loan_amnt
  2. term
  3. int_rate
  4. installment
  5. grade
  6. sub_grade
  7. emp_length
  8. home_ownership
  9. annual_inc
  10. verification_status
  11. purpose
  12. addr_state
  13. dti
  14. delinq_2yrs
  15. earliest_cr_line
  16. fico_range_low
  17. fico_range_high
  18. inq_last_6mths
  19. mths_since_last_delinq
  20. mths_since_last_record
  21. open_acc
  22. pub_rec
  23. revol_bal
  24. revol_util
  25. total_acc
  26. collections_12_mths_ex_med
  27. acc_now_delinq
  28. tot_coll_amt
  29. tot_cur_bal
  30. total_rev_hi_lim
  31. loan_status
  32. issue_d
  33. target
  34. emp_length_num
  35. earliest_cr_line_dt
  36. credit_history_years
  37. loan_to_income
  38. monthly_debt_burden
  39. fico_avg
  40. grade_num


In [43]:
# Data types summary
print("\nData Types:")
print(df_clean.dtypes.value_counts())


Data Types:
float64           27
object            10
int64              2
datetime64[ns]     1
Name: count, dtype: int64


In [44]:
# Quick statistics
df_clean.describe()

,loan_amnt,term,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,target,emp_length_num,earliest_cr_line_dt,credit_history_years,loan_to_income,monthly_debt_burden,fico_avg,grade_num
count,250000.000000,250000.000000,250000.000000,250000.000000,2.500000e+05,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,2.500000e+05,250000.000000,250000.000000,250000.000000,250000.000000,2.500000e+05,2.500000e+05,2.500000e+05,250000.000000,250000.000000,249995,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000
mean,14476.741900,41.842560,13.289079,439.770946,7.625523e+04,18.291751,0.318376,696.051240,700.051372,0.662060,32.652960,70.887652,11.583568,0.215600,1.634151e+04,51.773106,24.934796,0.017048,0.005160,2.520645e+02,1.380548e+05,3.240782e+04,0.214844,5.993668,1999-03-16 11:59:35.289105920,19.806823,5.600994,0.247242,698.051306,5.243144
min,900.000000,36.000000,5.310000,21.250000,0.000000e+00,0.000000,0.000000,630.000000,634.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,1.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.500000,1944-01-01 00:00:00,3.334247,0.000309,0.000123,632.000000,1.000000
25%,8000.000000,36.000000,9.750000,249.080000,4.600000e+04,11.800000,0.000000,670.000000,674.000000,0.000000,31.000000,71.000000,8.000000,0.000000,5.916000e+03,33.400000,16.000000,0.000000,0.000000,0.000000e+00,3.084300e+04,1.450000e+04,0.000000,3.000000,1995-05-01 00:00:00,14.509589,0.124996,0.046279,672.000000,4.000000
50%,12000.000000,36.000000,12.790000,375.490000,6.500000e+04,17.630000,0.000000,690.000000,694.000000,0.000000,31.000000,71.000000,11.000000,0.000000,1.111800e+04,52.100000,23.000000,0.000000,0.000000,0.000000e+00,8.020700e+04,2.410000e+04,0.000000,6.000000,2000-08-01 00:00:00,18.427397,0.199996,0.072265,692.000000,5.000000
75%,20000.000000,36.000000,16.020000,583.382500,9.000000e+04,24.090000,0.000000,710.000000,714.000000,1.000000,31.000000,71.000000,14.000000,0.000000,1.978025e+04,70.700000,32.000000,0.000000,0.000000,0.000000e+00,2.027202e+05,3.940000e+04,0.000000,10.000000,2004-07-01 00:00:00,23.684932,0.291664,0.105561,712.000000,6.000000
max,40000.000000,60.000000,30.990000,1691.280000,9.522972e+06,999.000000,21.000000,845.000000,850.000000,31.000000,180.000000,124.000000,79.000000,49.000000,2.560703e+06,193.000000,156.000000,9.000000,6.000000,6.214661e+06,4.772549e+06,9.999999e+06,1.000000,10.000000,2015-09-01 00:00:00,75.049315,40000.000000,1206.340000,847.500000,7.000000
std,8755.644942,10.299823,4.787245,262.706835,6.120769e+04,11.190356,0.875497,31.772112,31.772733,0.965122,15.577048,11.089954,5.483293,0.597647,2.325751e+04,24.534763,12.003923,0.142595,0.077701,1.270295e+04,1.551764e+05,3.860319e+04,0.410715,3.517703,NaN,7.629412,353.651971,10.649549,31.772422,1.297129


## 10. Save Cleaned Data

In [45]:
# Save to processed folder
output_path = '../data/processed/loan_cleaned.csv'
df_clean.to_csv(output_path, index=False)

print(f"✓ Data saved to: {output_path}")
print(f"✓ File size: {pd.read_csv(output_path).memory_usage(deep=True).sum() / 1e6:.2f} MB")

✓ Data saved to: ../data/processed/loan_cleaned.csv
✓ File size: 212.85 MB
